# Tier Selection

In [ ]:
import numpy as np
import os
import pandas as pd

### load tables

In [ ]:
ksad_p51 = pd.read_csv('/media/hcs-sci-psy-narun/abcd-data-release-5.1/core/mental-health/mh_p_ksads_ss.csv', low_memory=False )
# filtered_pv51_sum = ksad_p51[(ksad_p51['eventname'] == 'baseline_year_1_arm_1') & (ksad_p51['ksads_14_853_p'] == 1)]

In [ ]:
adhd_p51 = pd.DataFrame(pd.read_csv('/media/hcs-sci-psy-narun/abcd-data-release-5.1/core/mental-health/mh_p_ksads_adhd.csv', low_memory=False))
wisv_v51 = pd.DataFrame(pd.read_csv('/media/hcs-sci-psy-narun/abcd-data-release-5.1/core/neurocognition/nc_y_wisc.csv', low_memory=False))
ksad_y51 = pd.DataFrame(pd.read_csv('/media/hcs-sci-psy-narun/abcd-data-release-5.1/core/mental-health/mh_y_ksads_ss.csv', low_memory=False))
ksad_p51 = pd.DataFrame(pd.read_csv('/media/hcs-sci-psy-narun/abcd-data-release-5.1/core/mental-health/mh_p_ksads_ss.csv', low_memory=False))
bpm_t51 = pd.DataFrame(pd.read_csv('/media/hcs-sci-psy-narun/abcd-data-release-5.1/core/mental-health/mh_t_bpm.csv', low_memory=False))
cbcl_p51 = pd.DataFrame(pd.read_csv('/media/hcs-sci-psy-narun/abcd-data-release-5.1/core/mental-health/mh_p_cbcl.csv', low_memory=False))

### Baseline Full sample size from ksad table

In [ ]:
full_size = len(ksad_p51[(ksad_p51['eventname'] == 'baseline_year_1_arm_1')])
full_size
full_subkey = list(ksad_p51[(ksad_p51['eventname'] == 'baseline_year_1_arm_1')].src_subject_id)
len(full_subkey)

11747

### Tier 1 - v5.1
#### using scores from adhd table to calculate score equivalent to "ADHD Present on the KSADS COMP"

In [ ]:
column_names1 = [col for col in adhd_p51.columns if '404' in col or '409' in col or '414' in col ]
column_names2 = [col for col in adhd_p51.columns if any(x in col for x in map(str, range(973, 989)))]
column_names3 = [col for col in adhd_p51.columns if any(x in col for x in map(str, range(990, 995)))]
filtered_df1 = adhd_p51[adhd_p51[column_names1].isin([3, 4]).any(axis=1)]
filtered_df2 = filtered_df1[filtered_df1[column_names2].isin([2, 3, 4]).any(axis=1)]
count_condition = filtered_df2[column_names3].apply(lambda row: (row == 1).sum(), axis=1)
#Problem in at least 2 settings:
filtered_df3 = filtered_df2[count_condition >= 2] 
filtered_pv51 = filtered_df3[(filtered_df3['eventname'] == 'baseline_year_1_arm_1')]
T1_subkey = filtered_pv51.src_subject_id
len(T1_subkey)#/full_size

1034

### Tier 2 - v5.1
#### Subjects are filtered based on IQ, bipolar and psychosis in each respective table and subject list created.
#### 


In [ ]:
#IQ
filtered_iq_v51 = wisv_v51[((wisv_v51['pea_wiscv_tss'] > 3) & (wisv_v51['eventname'] == 'baseline_year_1_arm_1'))]
#bipolar:
p_column_names = [col for col in ksad_p51.columns if 'ksads_1_' in col and any(x in col for x in map(str, range(840, 848)))] #parent columns
c_column_names = [col for col in ksad_y51.columns if 'ksads_1_' in col and any(x in col for x in map(str, range(840, 848)))] #child columns

#bipolar - parent filtered:
filtered_p1 = ksad_p51[~(((ksad_p51[p_column_names].isin([1]).any(axis=1)) & (ksad_p51['ksads_2_833_p'] == 1)) | (ksad_p51['ksads_2_830_p'] == 1)) & (ksad_p51['eventname'] == 'baseline_year_1_arm_1')]
#bipolar - child filtered:
filtered_y1 = ksad_y51[~(((ksad_y51[c_column_names].isin([1]).any(axis=1)) & (ksad_y51['ksads_2_833_t'] == 1)) | (ksad_y51['ksads_2_830_t'] == 1)) & (ksad_y51['eventname'] == 'baseline_year_1_arm_1')]
#psychosis:
filtered_psy =  ksad_p51[~(ksad_p51['ksads_4_851_p'] == 1) & (ksad_p51['eventname'] == 'baseline_year_1_arm_1')]
#subjects who remained after filtering for IQ AND bipolar AND psychosis:
T2_subs = list((set(filtered_p1['src_subject_id']) & set(filtered_y1['src_subject_id'])) & set(filtered_psy['src_subject_id']) & set(filtered_iq_v51['src_subject_id']))
T2_finalsub = list(set(T1_subkey) & set(T2_subs))
len(T2_finalsub)#/full_size

944

### Tier 3 - v5.1
#### Subjects are filtered based on Teacher BPM attention scale

In [ ]:
filtered_bpm =  bpm_t51[(bpm_t51['bpm_t_scr_attention_t'] >= 65) & (bpm_t51['eventname'] == 'baseline_year_1_arm_1')]
T3_subkey = filtered_bpm.src_subject_id
T3_finalsub = list(set(T2_finalsub) & set(T3_subkey))
len(T3_finalsub)#/full_size

103

### Tier 4 - v5
#### Subjects are filtered based on parent CBCL Attention OR ADHD DSM5 scale

In [ ]:
filtered_cbclp =  cbcl_p51[((cbcl_p51['cbcl_scr_syn_attention_t'] >= 65) | (cbcl_p51['cbcl_scr_dsm5_adhd_t'] >= 65)) & (cbcl_p51['eventname'] == 'baseline_year_1_arm_1')]
T4_subkey = filtered_cbclp.src_subject_id
T4_finalsub = list(set(T3_finalsub) & set(T4_subkey))
len(T4_finalsub)#/full_size

/tmp/ipykernel_268207/2754702445.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_cbclp =  cbcl_p51[((cbcl_p5['cbcl_scr_syn_attention_t'] >= 65) | (cbcl_p51['cbcl_scr_dsm5_adhd_t'] >= 65)) & (cbcl_p51['eventname'] == 'baseline_year_1_arm_1')]


61

In [ ]:
#save:
pd.DataFrame(full_subkey).to_csv('/media/hcs-sci-psy-narun/ABCC_F/ksad/full_v51subkey.csv', index=False)
pd.DataFrame(T1_subkey).to_csv('/media/hcs-sci-psy-narun/ABCC_F/ksad/T1_v51subkey.csv', index=False)
pd.DataFrame(T2_finalsub).to_csv('/media/hcs-sci-psy-narun/ABCC_F/ksad/T2_v51finalsub.csv', index=False)
pd.DataFrame(T3_finalsub).to_csv('/media/hcs-sci-psy-narun/ABCC_F/ksad/T3_v51finalsub.csv', index=False)
pd.DataFrame(T4_finalsub).to_csv('/media/hcs-sci-psy-narun/ABCC_F/ksad/T4_v51finalsub.csv', index=False)